# Prepare VM with

- Fish
- Build Essential package
- Tmux

- IDA Pro
- GDB
    - GEF
    - GDB Dashboard
- Radare2

- Radamsa
- Libfuzzer?
- AFL?


# Protections

#### Stack protector (Canaries)

#### ASLR  
- Randomizes the addresses of the stack, heap and shared libraries  
- Can't use hardcoded addreses, can't do ret2libc
- Bypass:
- - The offset into ithe libraries is still known. If you can figure out the base address for the library is game over
- - Use procedural Linkage Table
- - Overwrite the Global Offset Table
- - Use Brute-force (as a last resort)  

##### PLT = procedural linkage table  
For shared libraries, the .text segment is shared between proceses but the data segment is not common between programs.  
The code in libraries is position independant (all relative jumps)  
When a library function is called, a stub in the PLT is called.  
The stubs looks up the address into the GOT (global offset table)  
The PLT contains all the entries for the functions that the program uses (this is the table of shared functions that the linker writes so it adds only the functions the executable needs).  
As addreses on the PLT are part of the executable, they ARE NOT RANDOMIZED!!  
You can build a chain of libC functions with this.  
The only limitation is that only functions linked (used) in the binary are present in the PLT.  

##### GOT = global object table
Stores location of library functions  
Two ways to attack GOT:  
- Overwrite the entries in the table  
- Dereference: store offset in a register  
To locate GOT entries:
`objdump -R <program>`  

> How to GOT overwirte:  
- Find offsets for a called function and desired function:  
`readelf -s <library>`  
- Calculate offset difference  
`offset_delta = <desired addr> - <called addr>`  
- Overwrite GOT  
`GOT[<called>] = GOT[<called>] + offset_delta` 
- Make everything usin ROP to also bypass DEP  

After having overwriten the GOT, every time someone calls the overriden function it will actually call the new one  

> How to GOT dereference:  
- Find offset as before  
- Calculate delta as before  
- Load address into a register (eax used as example)  
` eax = GOT[<called>]  
  eax = eax + <offset_delta>  `  
- Make everything usin ROP to also bypass DEP  
`  
stack grows downwards  
v ---------------------------------------------------
| gadget4 => etc, etc, etc
| param1  => 1st param for libc.execve()
| param2  => 1st param for libc.execve()
| param3  => 1st param for libc.execve()
| gadget3 => i.e this is a call to libc.execve()
| gedget2 => rop gadget preparing for libc.execve()
| gadget1 => rop gadget preparing for libc.execve()  
| ---------------------------------------------------
v  
stack grows downwards
`    
> How to calculate GOT offset  
- Locate GOT offset present in the binary that I want to overwrite or derefence  
`objdump -R <program>`  
`0804a00c  JUMP  strcpy@GLIBC_2.0`  
- Locate the offsets for the function I want to overwrite or derefence and the function I want to call  
`readelf -s /lib/i386-linux-gnu/libc-2.27.so | grep -E "strcpy@@GLIBC_2|system@@GLIBC_2" | column -t`  
`  108:   0007e370  59  IFUNC  GLOBAL  DEFAULT  13  strcpy@@GLIBC_2.0  
   1510:  0003d200  55  FUNC   WEAK    DEFAULT  13  system@@GLIBC_2.0  `  
- For GOT overwrite: set GOT entry from 1st step to current value + delta between strcpy and system in the library (from step 2)  
- For GOT derefence load into a register the value for str in the GOT, then add the delta between the two funtions in the library. Finally call the register value.  

#### DEP  

##### Chaining libC function calls  
Search for libC functions:  
`readelf -s /lib32/libc-2.23.so | grep sprintf`  
Exact function address = libC function addres in memory + offset obtained using the command above  
It works if ASLR is disabled. Using this we can bypass DEP/W^X by changing the stack x permissions. We can also use this to escalate privileges.  

#### Stack Canaries  
Compiler inserts extra code that checks for the canary  
There are two types of common used canaries:
- Terminator  
    - Contains characters that are incompatible with functions like strcpy
    - A good terminator look like: 0x000a0dff == null + \n + \r + 0xff (sometimes is threated as EOF)
    - Is a static value, can be search for and you can easily create a new stack with the correct stack canary on it
- Random  
    - It does not have bad chars
    - It's a random value that you cannot easily prevent
    - Much harder to bypass than the terminator canary (depending on how random the value is)
    
    
### HEAP  

Ptmalloc is the current implementation used by gcc (need to check this)  




